# ML Surrogates for Chemical Processes with OMLT
This notebook illustrates the use of TensorFlow Keras and OMLT to produce an ML surrogate based on data from a chemical process flowsheet.

There are several reasons to build surrogate models for complex processes, even when higher fidelity models already exist (e.g., reduce model size, improve convergence reliability, replace models with externally compiled code and make them fully-equation oriented).

In this example, we have an existing model for an auto-thermal reformer flowsheet that has been built using the IDAES-PSE package. IDAES-PSE is a Python package that is built on Pyomo and provides a framework for equation-oriented modeling and analysis of advanced energy systems. We use this package to generate data for our systems, and then we utilize this data in an optimization problem with OMLT. To learn more about IDAES, see the [IDAES-PSE Github Page](https://github.com/IDAES/IDAES-PSE) or [IDAES Read-the-docs](https://idaes-pse.readthedocs.io/en/stable/).

## The Auto-thermal Reformer Process

The figure below shows the reformer process as modeled in IDAES.

![Reformer Flowsheet](https://github.com/joelwwiggins/supply_chain_simulation/blob/main/nn_optimization/images/reformer.png?raw=1)

This model has 12 outputs of interest, the steam flowrate, the reformer duty, and the properties of the outlet stream, including temperature, pressure, and composition. We are interested modeling how these outputs change as a function of two operating (or input) variables: 
- the fraction of natural gas that bypasses the reformer
- steam to natural gas flow ratio

We have already used IDAES to generate a CSV file that contains the input and output data for 2800 data points for our system.

In this example, we will train a neural network model with sigmoid activations from our process data and then demonstrate that we can solve an optimization problem with that surrogate model. In realistic applications, this surrogate model would form part of a design or operations problem with a much larger flowsheet. 

In [2]:
%pip install omlt
%pip install onnx
%pip install onyx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 20.8 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 13.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 99 kB 582 kB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 177 kB 30.5 MB/s 
     |████████████████████████████████| 176 kB 34.3 MB/s 
     |████████████████████████████████| 93 kB 1.9 MB/s 
     |████████████████████████████████| 93 kB 1.7 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 91 kB 10.2 MB/s 
     |███████████████

In [3]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress CUDA warnings from tensorflow

# import the necessary packages
from omlt import OmltBlock, OffsetScaling
from omlt.io.keras_reader import load_keras_sequential
from omlt.neuralnet import FullSpaceSmoothNNFormulation
import pyomo.environ as pyo
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [11]:
# read in our csv data
columns = ['Bypass Fraction', 'NG Steam Ratio', 'Steam Flow',
           'Reformer Duty','AR', 'C2H6', 'C3H8', 'C4H10',
           'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2']
df = pd.read_csv(r'C:\Users\jwiggins\Documents\Python_virtual_env\Supply Chain Simulation\supply_chain_simulation\supply_chain_simulation\nn_optimization\reformer.csv', usecols=columns)
print(df)

      Bypass Fraction  NG Steam Ratio  Steam Flow  Reformer Duty        AR  \
0                 0.8        0.800000    0.193898    9806.732716  0.002662   
1                 0.8        0.810526    0.196449    9846.047501  0.002660   
2                 0.8        0.821053    0.199000    9885.419259  0.002657   
3                 0.8        0.831579    0.201552    9924.849127  0.002654   
4                 0.8        0.842105    0.204103    9964.338177  0.002651   
...               ...             ...         ...            ...       ...   
2795              0.1        1.157895    1.262887   39771.876388  0.004086   
2796              0.1        1.168421    1.274368   39989.582852  0.004080   
2797              0.1        1.178947    1.285849   40207.531167  0.004073   
2798              0.1        1.189474    1.297330   40425.721366  0.004067   
2799              0.1        1.200000    1.308811   40644.153425  0.004060   

          C2H6      C3H8     C4H10       CH4        CO       CO

In [21]:
# separate the data into inputs and outputs
inputs = ['Bypass Fraction', 'NG Steam Ratio']
outputs = [ 'Steam Flow', 'Reformer Duty','AR', 'C2H6', 'C3H8', 'C4H10',
           'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2']
dfin = df[inputs]
dfout = df[outputs]

In [22]:
# We scale the data for improved training, however, we want to formulate
# our optimizaton problem on the original variables. Therefore, we keep
# the scaling parameters to use later in our optimization formulation

x_offset, x_factor = dfin.mean().to_dict(), dfin.std().to_dict()
y_offset, y_factor = dfout.mean().to_dict(), dfout.std().to_dict()

dfin = (dfin - dfin.mean()).divide(dfin.std())
dfout = (dfout - dfout.mean()).divide(dfout.std())

# capture the minimum and maximum values of the scaled inputs
# so we don't use the model outside the valid range
scaled_lb = dfin.min()[inputs].values
scaled_ub = dfin.max()[inputs].values

In [23]:
# create our Keras Sequential model
nn = Sequential(name='reformer_sigmoid_4_20')
nn.add(Dense(units=20, input_dim=len(inputs), activation='sigmoid'))
nn.add(Dense(units=20, activation='sigmoid'))
nn.add(Dense(units=20, activation='sigmoid'))
nn.add(Dense(units=20, activation='sigmoid'))
nn.add(Dense(units=len(outputs)))
nn.compile(optimizer=Adam(), loss='mse')

In [24]:
# train our model
x = dfin.values
y = dfout.values

history = nn.fit(x, y, epochs=100)

Epoch 1/100
88/88 [==============================] - 0s 2ms/step - loss: 1.0311
Epoch 2/100
88/88 [==============================] - 0s 2ms/step - loss: 0.9941
Epoch 3/100
88/88 [==============================] - 0s 2ms/step - loss: 0.9573
Epoch 4/100
88/88 [==============================] - 0s 2ms/step - loss: 0.6406
Epoch 5/100
88/88 [==============================] - 0s 2ms/step - loss: 0.2066
Epoch 6/100
88/88 [==============================] - 0s 2ms/step - loss: 0.1445
Epoch 7/100
88/88 [==============================] - 0s 1ms/step - loss: 0.1237
Epoch 8/100
88/88 [==============================] - 0s 2ms/step - loss: 0.1093
Epoch 9/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0993
Epoch 10/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0918
Epoch 11/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0860
Epoch 12/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0817
Epoch 13/100
88/88 [=================

In [17]:
# save the model to disk
# While not technically necessary, this shows how we can load a previously saved model into
# our optimization formulation)
nn.save('reformer_nn')

INFO:tensorflow:Assets written to: reformer_nn\assets


## Optimization Problem
In this small example, we will formulate a simple optimization problem that seeks to maximize the concentration of Hydrogen in the outlet while placing an upper bound on the Nitrogen concentration.

In [12]:
# first, create the Pyomo model
m = pyo.ConcreteModel()

In [13]:
# create the OmltBlock to hold the neural network model
m.reformer = OmltBlock()

In [14]:
# load the Keras model
nn_reformer = keras.models.load_model('reformer_nn', compile=False)

# Note: The neural network is in the scaled space. We want access to the
# variables in the unscaled space. Therefore, we need to tell OMLT about the
# scaling factors
scaler = OffsetScaling(
        offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
        factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
        offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
        factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))}
    )

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

# create a network definition from the Keras model
net = load_keras_sequential(nn_reformer, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds)

# create the variables and constraints for the neural network in Pyomo
m.reformer.build_formulation(FullSpaceSmoothNNFormulation(net))

In [15]:
# now add the objective and the constraints
h2_idx = outputs.index('H2')
n2_idx = outputs.index('N2')
m.obj = pyo.Objective(expr=m.reformer.outputs[h2_idx], sense=pyo.maximize)
m.con = pyo.Constraint(expr=m.reformer.outputs[n2_idx] <= 0.34)

In [16]:
# now solve the optimization problem
solver = pyo.SolverFactory('ipopt')
status = solver.solve(m, tee=True)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1812
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:       80

Total number of variables............................:      214
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      190
                     variables with only upper bounds:        0
Tot

In [17]:
print('Bypass Fraction:', pyo.value(m.reformer.inputs[0]))
print('NG Steam Ratio:', pyo.value(m.reformer.inputs[1]))
print('H2 Concentration:', pyo.value(m.reformer.outputs[h2_idx]))
print('N2 Concentration:', pyo.value(m.reformer.outputs[n2_idx]))

Bypass Fraction: 0.10000023663266308
NG Steam Ratio: 1.1139330924434168
H2 Concentration: 0.3313810317960735
N2 Concentration: 0.34000000435466543


In [29]:
y_pred=pd.DataFrame(nn.predict(dfin))
df_df=pd.DataFrame(dfout)
cols=df_df.columns
y_pred.columns=cols
y_pred

,Steam Flow,Reformer Duty,AR,C2H6,C3H8,C4H10,CH4,CO,CO2,H2,H2O,N2
0,-1.805541,-1.683807,-2.222771,2.422867,2.425049,2.429208,2.450698,-1.828085,-2.701060,-2.362022,-2.619916,-2.218774
1,-1.798042,-1.680849,-2.231698,2.423327,2.425778,2.428618,2.449987,-1.849774,-2.673985,-2.365599,-2.585436,-2.228149
2,-1.790231,-1.677695,-2.240525,2.423472,2.426250,2.427757,2.449010,-1.871596,-2.646191,-2.368973,-2.550155,-2.237422
3,-1.782122,-1.674349,-2.249238,2.423302,2.426465,2.426629,2.447771,-1.893510,-2.617738,-2.372141,-2.514146,-2.246578
4,-1.773732,-1.670812,-2.257820,2.422820,2.426425,2.425242,2.446273,-1.915475,-2.588687,-2.375098,-2.477486,-2.255595
...,...,...,...,...,...,...,...,...,...,...,...,...
2795,2.166839,2.045166,0.961004,-1.220388,-1.206330,-1.205281,-1.217341,0.566930,1.731788,1.105814,1.831165,0.955730
2796,2.198785,2.066802,0.947557,-1.219256,-1.204666,-1.203916,-1.217012,0.535918,1.765291,1.100016,1.877622,0.941576
2797,2.229663,2.087800,0.934395,-1.218041,-1.202916,-1.202438,-1.216642,0.505735,1.797619,1.094322,1.922605,0.927739
2798,2.259445,2.108130,0.921532,-1.216750,-1.201088,-1.200856,-1.216229,0.476410,1.828756,1.088740,1.966075,0.914235


In [30]:
heat_map_data=pd.concat([y_pred,df_df]).corr()
heat_map_data

,Steam Flow,Reformer Duty,AR,C2H6,C3H8,C4H10,CH4,CO,CO2,H2,H2O,N2
Steam Flow,1.000000,0.991731,0.862744,-0.919465,-0.919499,-0.919442,-0.919637,0.736538,0.951606,0.898241,0.909995,0.859710
Reformer Duty,0.991731,1.000000,0.903909,-0.942073,-0.942080,-0.942050,-0.942183,0.803832,0.917788,0.928771,0.855950,0.901701
AR,0.862744,0.903909,1.000000,-0.987657,-0.987664,-0.987673,-0.987541,0.971616,0.812324,0.995851,0.696998,0.999971
C2H6,-0.919465,-0.942073,-0.987657,1.000000,0.999982,0.999993,0.999985,-0.922607,-0.893593,-0.997789,-0.800649,-0.986539
C3H8,-0.919499,-0.942080,-0.987664,0.999982,1.000000,0.999991,0.999988,-0.922612,-0.893588,-0.997802,-0.800666,-0.986540
C4H10,-0.919442,-0.942050,-0.987673,0.999993,0.999991,1.000000,0.999986,-0.922623,-0.893576,-0.997803,-0.800637,-0.986548
CH4,-0.919637,-0.942183,-0.987541,0.999985,0.999988,0.999986,1.000000,-0.922332,-0.893926,-0.997756,-0.801105,-0.986423
CO,0.736538,0.803832,0.971616,-0.922607,-0.922612,-0.922623,-0.922332,1.000000,0.651337,0.946087,0.507664,0.973256
CO2,0.951606,0.917788,0.812324,-0.893593,-0.893588,-0.893576,-0.893926,0.651337,1.000000,0.861987,0.984241,0.808222
H2,0.898241,0.928771,0.995851,-0.997789,-0.997802,-0.997803,-0.997756,0.946087,0.861987,1.000000,0.759295,0.995191


In [35]:
import plotly.express as px
fig=px.imshow(heat_map_data)

fig.show()